# Conformance Checking with a LTLf model

The class `ProcessMiningTasks.ConformanceChecking.LTLAnalyzer.LTLAnalyzer` provides a way to check if the log conforms to a Linear Temporal Logic on finite traces (LTLf) formula. The formula can be a provided by the user as a string, note that we adopted the LTLf syntax [here](http://ltlf2dfa.diag.uniroma1.it/ltlf_syntax). In addition, we also provide the following set of LTLf templates:

1. `eventually_a`;
2. `next_a`;
3. `eventually_a_then_b`;
4. `eventually_a_or_b`;
5. `eventually_a_next_b`;
6. `eventually_a_then_b_then_c`;
7. `eventually_a_next_b_next_c`;

and the following [Target-Branched DECLARE templates](https://www.sciencedirect.com/science/article/pii/S0306437915001271):

1. `precedence`;
2. `chain_precedence`;
3. `responded_existence`;
4. `chain_response`;
5. `not_chain_precedence`;
6. `not_chain_response`;
7. `response`;
8. `not_precedence`;
9. `not_response`;
10. `not_responded_existence`;
11. `alternate_response`;
12. `alternate_precedence`.

First of all, an event log has to be imported.

In [44]:
import sys
import os
import pathlib

SCRIPT_DIR = pathlib.Path("../../../", "src").resolve()
sys.path.append(os.path.dirname(SCRIPT_DIR))

from src.Declare4Py.D4PyEventLog import D4PyEventLog

log_path = os.path.join("../../../", "tests", "test_logs","Sepsis Cases.xes.gz")
event_log = D4PyEventLog()
event_log.parse_xes_log(log_path)

parsing log, completed traces ::   0%|          | 0/1050 [00:00<?, ?it/s]

The next step is to create an `LTLModel` from an input LTLf formula in string format with the class `ProcessModels.LTLModel.LTLModel` and pass it to the LTL conformance checker implemented in the `src.Declare4Py.ProcessMiningTasks.ConformanceChecking.LTLAnalyzer.LTLAnalyzer` class. The corresponding method `run` will do the conformance checking and return a Pandas dataframe. This dataframe contains the traces ids in the first column and the results of the conformance checking in the second column. The `run` method transforms the LTLf formula into a Deterministic Finite state Automaton (DFA) and checks whether a trace in a log is accepted. This transformation is performed by using two backends:

 - Lydia, C++ backend that needs to be installed with Docker, more details [here](https://github.com/whitemech/logaut/tree/main);
 - LTLf2DFA, that needs to be installed with `pip install git+https://github.com/whitemech/LTLf2DFA.git@develop#egg=ltlf2dfa`. More details [here](https://github.com/whitemech/LTLf2DFA/).

As explained in the tutorial regarding process models, it is possible to switch the backends of the LTLf model with the `to_ltlf2dfa_backend` and `to_lydia_backend` methods. The default backend is Lydia.

For speeding up the computation, the `run` method takes as input the integer parameter `jobs` that sets the number of processes to run in parallel. Each process considers a portion of the input event log. The default value of the number of jobs is 1. Note that, with small logs (i.e., with a small number of events) the performance with multiple jobs can be comparable with the ones of a single job.

We show an example of LTLf conformance checking with the `F(ER Triage)` formula.

In [45]:
from src.Declare4Py.ProcessModels.LTLModel import LTLModel
from src.Declare4Py.ProcessMiningTasks.ConformanceChecking.LTLAnalyzer import LTLAnalyzer

model = LTLModel()
model.parse_from_string("F(CRP)")

analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run(jobs=2)
conf_check_res_df

,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


## Conformance Checking with LTLf Templates

Declare4Py offers some LTLf templates in the `src.Declare4Py.ProcessModels.LTLModel.LTLTemplate` class. You just need to instantiate a single template by passing the template name to the `LTLTemplate` class and then filling the template with some proper activites to obtain an `LTLModel` and run the LTL checker.

The `LTLTemplate`'s function `fill_template` expects a list with the parameters equal to those expected by the template and returns an `LTLModel` object containing the parsed formula of the template. The LTLf formula of the template can be retrieved by accessing the `formula` attribute of the `LTLModel` object.

### `eventually_a`

This is a unary template taking one activity as input. The corresponding LTLf formula is `F(a)`.

In [46]:
from src.Declare4Py.ProcessModels.LTLModel import LTLTemplate

template: LTLTemplate = LTLTemplate('eventually_a')
model: LTLModel = template.fill_template(['CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: F(crp)


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `next_a`
This is a unary template taking one activity as input. The corresponding LTLf formula is `X(a)`.

In [47]:
template = LTLTemplate('next_a')
model = template.fill_template(['ER Triage'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: X(ertriage)


,case:concept:name,accepted
0,A,False
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,True
1047,JNA,True
1048,KNA,True


### `eventually_a_then_b`

This is a binary template taking two activities as input. The corresponding LTLf formula is `F(a && F(b))`.

In [48]:
template = LTLTemplate('eventually_a_then_b')
model = template.fill_template(['Leucocytes', 'CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: F(leucocytes && F(crp))


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,False
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `eventually_a_or_b`

This is a binary template taking two activities as input. The corresponding LTLf formula is `F(a) || F(b)`.

In [49]:
template = LTLTemplate('eventually_a_or_b')
model = template.fill_template(['Leucocytes', 'CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: F(leucocytes) || F(crp)


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `eventually_a_next_b`

This is a binary template taking two activities as input. The corresponding LTLf formula is `F(a && X(b))`.

In [50]:
template = LTLTemplate('eventually_a_next_b')
model = template.fill_template(['Leucocytes', 'CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: F(leucocytes && X(crp))


,case:concept:name,accepted
0,A,True
1,B,False
2,C,True
3,D,True
4,E,False
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `eventually_a_then_b_then_c`

This is a ternary template taking three activities as input. The corresponding LTLf formula is `F(a && F(b && F(c)))`.

In [51]:
template = LTLTemplate('eventually_a_then_b_then_c')
model = template.fill_template(['ER Registration', 'Leucocytes', 'CRP'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: F(erregistration && F(leucocytes && F(crp)))


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,False
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `eventually_A_next_B_next_C`

This is a ternary template taking three activities as input. The corresponding LTLf formula is `F(a && X(b && X(c)))`.

In [52]:
template = LTLTemplate('eventually_a_next_b_next_c')
model = template.fill_template(['ER Registration', 'CRP', 'Leucocytes'])
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: F(erregistration && X(crp && X(leucocytes)))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,False
1046,INA,False
1047,JNA,False
1048,KNA,False


## Conformance Checking with Target-Branched DECLARE templates

For these binary templates, the `fill_template` method takes two lists of activities (in string format) as parameters. For each template, the instantiated template formula and its corresponding checking in the input log will be computed.

### `precedence`

In [53]:
template = LTLTemplate('precedence')
activities_a = ["ER Triage", "CRP"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: ((!(leucocytes || admissionnc || releasea))U(ertriage || crp)) || G(!(leucocytes || admissionnc || releasea))


,case:concept:name,accepted
0,A,False
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,True
1047,JNA,True
1048,KNA,True


### `chain_precedence`

In [54]:
template = LTLTemplate('chain_precedence')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: G(X(leucocytes || admissionnc || releasea) ->  (ertriage || crp || leucocytes))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,False
1046,INA,False
1047,JNA,False
1048,KNA,False


### `responded_existence`

In [55]:
template = LTLTemplate('responded_existence')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: F(ertriage || crp || leucocytes) -> F(leucocytes || admissionnc || releasea)


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `chain_response`

In [56]:
template = LTLTemplate('chain_response')
activities_a = ["ER Registration", "CRP"]
activities_b = ["ER Triage", "Leucocytes", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: G((erregistration  || crp) -> X(ertriage || leucocytes || releasea))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,True
...,...,...
1045,HNA,False
1046,INA,True
1047,JNA,True
1048,KNA,False


### `not_chain_precedence`

In [57]:
template = LTLTemplate('not_chain_precedence')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: G(X(leucocytes || admissionnc || releasea) ->  !(ertriage || crp || leucocytes))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,False
1046,INA,True
1047,JNA,True
1048,KNA,False


### `not_chain_response`

In [58]:
template = LTLTemplate('not_chain_response')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: G((ertriage || crp || leucocytes) -> X(!(leucocytes || admissionnc || releasea)))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,False
1046,INA,True
1047,JNA,True
1048,KNA,False


### `response`

In [59]:
template = LTLTemplate('response')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: G((ertriage || crp || leucocytes) -> F(leucocytes || admissionnc || releasea))


,case:concept:name,accepted
0,A,True
1,B,True
2,C,True
3,D,True
4,E,True
...,...,...
1045,HNA,True
1046,INA,False
1047,JNA,False
1048,KNA,True


### `not_precedence`

In [60]:
template = LTLTemplate('not_precedence')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: G(F(leucocytes || admissionnc || releasea) -> !(ertriage || crp || leucocytes))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,False
1046,INA,True
1047,JNA,True
1048,KNA,False


### `not_response`

In [61]:
template = LTLTemplate('not_response')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: G((ertriage || crp || leucocytes) ->  !(F(leucocytes || admissionnc || releasea)))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,False
1046,INA,True
1047,JNA,True
1048,KNA,False


### `not_responded_existence`

In [62]:
template = LTLTemplate('not_responded_existence')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: F(ertriage || crp || leucocytes) ->  !(F(leucocytes || admissionnc || releasea))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,False
1046,INA,True
1047,JNA,True
1048,KNA,False


### `alternate_response`

In [63]:
template = LTLTemplate('alternate_response')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: G((ertriage || crp || leucocytes) -> X((!(ertriage || crp || leucocytes)U(leucocytes || admissionnc || releasea))))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,False
1046,INA,False
1047,JNA,False
1048,KNA,False


### `alternate_precedence`

In [64]:
template = LTLTemplate('alternate_precedence')
activities_a = ["ER Triage", "CRP", "Leucocytes"]
activities_b = ["Leucocytes", "Admission NC", "Release A"]
model = template.fill_template(activities_a, activities_b)
print(f"Formula: {model.formula}")
analyzer = LTLAnalyzer(event_log, model)
conf_check_res_df = analyzer.run()
conf_check_res_df

Formula: ((!(leucocytes || admissionnc || releasea))U(ertriage || crp || leucocytes)) && G((leucocytes || admissionnc || releasea) -> X((!(leucocytes || admissionnc || releasea))U(ertriage || crp || leucocytes)) || G(!(leucocytes || admissionnc || releasea)))


,case:concept:name,accepted
0,A,False
1,B,False
2,C,False
3,D,False
4,E,False
...,...,...
1045,HNA,True
1046,INA,True
1047,JNA,True
1048,KNA,True
